In [ ]:
import requests
import keyring


def create_token(): 
    global access_token
    url = "https://accounts.spotify.com/api/token"

    payload = f'grant_type=client_credentials&client_id={keyring.get_password("Spotify", "client_id")}&client_secret={keyring.get_password("Spotify", "client_secret")}'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Cookie': '__Host-device_id=AQD5YwRrHqtWCnT9TIwGJ0EN2j2QZ0CU__xVNmGEdFaBVyn3GGwnDSRXxfdsqLFvWgbemBlHtvqE2JJJfhg5880FF3ZwqPZI83k; sp_tr=false'
    }

    response = requests.request("POST", url, headers=headers, data=payload, timeout=200)
    access_token = response.json()['access_token']

create_token()

In [ ]:
import requests

url = "https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, timeout=200)

print(response.text)


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Autenticación con la API de Spotify
client_credentials_manager = SpotifyClientCredentials(client_id='bf6ceac0ca1548b994512ab886eff7ac', client_secret='5570675867f94eef84f6504ac61b5dde')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:


# Lista de 10 artistas (puedes agregar los artistas que prefieras)
my_10_artists = [
    'Mumford And Sons', 'Nirvana', 'Royel Otis', 'Creedence Clearwater Revival', 'The Blaze', 'Bob Dylan', 'The Doors', 'Fleetwood Mac', 'Phill Collins', 'The War on Drugs'
]

# Imprime la lista para verificar
print(my_10_artists)

In [ ]:
# Función para obtener el ID de un artista por su nombre
def get_artist_id(artist_name):
    results = sp.search(q='artist:' + artist_name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]['id']
    else:
        return None

In [ ]:
# Función para obtener las primeras 50 apariciones de un artista
def get_first_50_appearances(artist_name):
    artist_id = get_artist_id(artist_name)
    if artist_id is None:
        return []
    
    results = sp.artist_top_tracks(artist_id, country='US')
    tracks = results['tracks']
    appearances = []

    for track in tracks[:50]:  # Limit to 50 tracks
        appearances.append({
            'name': track['name'],
            'album': track['album']['name'],
            'release_date': track['album']['release_date']
        })
    return appearances

In [ ]:
# Función principal que toma una lista de nombres de artistas y devuelve un diccionario con sus primeras 50 apariciones
def get_artists_appearances(artists_list):
    appearances_dict = {}
    for artist in artists_list:
        appearances_dict[artist] = get_first_50_appearances(artist)
    return appearances_dict

# Obtener las primeras 50 apariciones de los 20 artistas
my_10_artists_appearances = get_artists_appearances(my_10_artists)

# Imprimir los resultados para verificar
for artist, appearances in my_10_artists_appearances.items():
    print(f"Artista: {artist}")
    for appearance in appearances:
        print(f"  - Canción: {appearance['name']}, Álbum: {appearance['album']}, Fecha de lanzamiento: {appearance['release_date']}")

In [ ]:
def get_playlist_tracks(playlist_id):
    """Obtiene todas las canciones de una lista de reproducción dado su ID."""
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
    

In [ ]:
def get_artist_albums(artist_id):
    """Obtiene todos los álbumes de un artista dado su ID."""
    albums = []
    results = sp.artist_albums(artist_id, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    return albums

def get_album_tracks(album_id):
    """Obtiene todas las canciones de un álbum dado su ID."""
    tracks = []
    results = sp.album_tracks(album_id)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
def get_audio_features(track_ids):
    """Obtiene las características de audio de una lista de IDs de canciones."""
    features = []
    # Spotify API has a limit of 100 ids per request
    for i in range(0, len(track_ids), 100):
        audio_features = sp.audio_features(tracks=track_ids[i:i+100])
        features.extend(audio_features)
    return features

In [ ]:
def collect_songs_and_features_from_playlist(playlist_id):
    """Recopila canciones y sus características de audio desde una lista de reproducción."""
    track_ids = []
    tracks = get_playlist_tracks(playlist_id)
    
    for item in tracks:
        track = item['track']
        if track:
            track_ids.append(track['id'])
    
    features = get_audio_features(track_ids)
    
    songs_data = []
    for i, track in enumerate(tracks):
        if track['track']:
            song_info = {
                'name': track['track']['name'],
                'artist': [artist['name'] for artist in track['track']['artists']],
                'id': track['track']['id'],
                'features': features[i]
            }
            songs_data.append(song_info)
    
    return songs_data


def collect_songs_from_artists_in_playlist(playlist_id):
    """Recopila canciones de artistas presentes en una lista de reproducción."""
    tracks = get_playlist_tracks(playlist_id)
    artist_ids = {artist['id'] for item in tracks for artist in item['track']['artists'] if item['track']}
    
    all_tracks = []
    for artist_id in artist_ids:
        albums = get_artist_albums(artist_id)
        for album in albums:
            album_tracks = get_album_tracks(album['id'])
            all_tracks.extend(album_tracks)
    
    track_ids = [track['id'] for track in all_tracks if track]
    features = get_audio_features(track_ids)
    
    songs_data = []
    for i, track in enumerate(all_tracks):
        if track:
            song_info = {
                'name': track['name'],
                'artist': [artist['name'] for artist in track['artists']],
                'id': track['id'],
                'features': features[i]
            }
            songs_data.append(song_info)
    
    return songs_data

In [ ]:
playlist_id = '37i9dQZF1E8KX8LprsFuoI'  # ID de la lista "Today's Top Hits" 

# Recopilar canciones y características de la lista de reproducción
songs_data_from_playlist = collect_songs_and_features_from_playlist(playlist_id)

# Recopilar canciones y características de los artistas presentes en la lista de reproducción
songs_data_from_artists = collect_songs_from_artists_in_playlist(playlist_id)

# Combinamos los datos de ambas recopilaciones
all_songs_data = songs_data_from_playlist + songs_data_from_artists

# Imprimir el número total de canciones recopiladas
print(f"Total de canciones recopiladas: {len(all_songs_data)}")

print(all_songs_data)



In [ ]:
import pandas as pd

In [ ]:
playlist_df=pd.DataFrame(all_songs_data)

In [ ]:
playlist_df